In [449]:
import pandas as pd
import numpy as np

In [511]:
hosp = pd.read_csv('Hospitals_geocoded.csv')

#### Section Used to Manually Obtain Latitude and Longitude for Hospitals
* no longer need to be run
---------------------------------------------------------------------------

In [22]:
# hosp_missing = hosp[hosp.latitude.isna()][['Facility Name','Address','City']].reset_index().copy()
# hosp_missing['latitude'] = [np.NaN for n in range(hosp_missing.shape[0])]
# hosp_missing['longitude'] = [np.NaN for n in range(hosp_missing.shape[0])]

# hosp_missing = pd.read_csv('temp_latlong_filling.csv', index_col='Facility ID')
# missing = hosp_missing['Facility Name']+', '+hosp_missing.Address+', '+hosp_missing.City+', USA'

In [4]:
# import geopy
# from geopy import Nominatim

# locator = Nominatim(user_agent="nycDSA")

In [420]:
# current = 88
# missing[current]

'William R Sharpe, Jr Hospital, 936 SHARPE HOSPITAL ROAD, WESTON, USA'

In [422]:
# lat = locator.geocode("William R. Sharpe Hospital, WESTON, USA")[1][0]
# long = locator.geocode("William R. Sharpe Hospital, WESTON, USA")[1][1]
# print(f'{lat}, {long}')

39.0408843, -80.4961853


In [423]:
# hosp_missing.loc[current,'latitude'] = lat
# hosp_missing.loc[current,'longitude'] = long

In [424]:
# hosp_missing.loc[[current]]

,Facility ID,Facility Name,Address,City,latitude,longitude
88,514012,"William R Sharpe, Jr Hospital",936 SHARPE HOSPITAL ROAD,WESTON,39.040884,-80.496185



---------------------------------------------------------------------------

## 🏥 Quick EDA

In [512]:
hosp.columns

Index(['Facility_Name', 'Address', 'City', 'State', 'ZIP_Code', 'County_Name',
       'Phone_Number', 'Hospital_Type', 'Hospital_Ownership',
       'Emergency_Services', 'Overall_Rating'],
      dtype='object')

In [513]:
print('-----------------------------------')
print(hosp.Hospital_Type.value_counts())
print('\n')
print('-----------------------------------')
print(hosp.Emergency_Services.value_counts())
print('\n')

-----------------------------------
Acute Care Hospitals                  553
Critical Access Hospitals              92
Psychiatric                            83
Acute Care - Department of Defense     12
Childrens                              11
Name: Hospital_Type, dtype: int64


-----------------------------------
Yes    640
No     111
Name: Emergency_Services, dtype: int64




#### Remove <span style="color:darkred">Psychiatric</span> & <span style="color:red">non-ER</span> Hospitals

In [514]:
hosp = hosp[(hosp.Hospital_Type!='Psychiatric') & (hosp.Emergency_Services=='Yes')]

In [517]:
print('-----------------------------------')
print(f'Total ZIP codes: {hosp.ZIP_Code.nunique()}')
print('\n')
print('-----------------------------------')
print('# Hospitals by State')
print(hosp.State.value_counts())
print('\n')
print('-----------------------------------')
print(hosp.Overall_Rating.value_counts())

-----------------------------------
Total ZIP codes: 611


-----------------------------------
# Hospitals by State
FL    170
GA    126
NC    103
VA     80
SC     55
WV     46
MD     43
DC      6
Name: State, dtype: int64


-----------------------------------
3                175
4                139
2                136
Not Available     94
5                 52
1                 33
Name: Overall_Rating, dtype: int64


## Stanley Martin Homes Target ZIP codes and STATES

In [522]:
smh = pd.read_csv('redfin_scrapy/smh_zipcodes.csv', index_col='ZipCode')
smh.State.value_counts()

FL    1464
VA    1213
NC    1079
GA     950
WV     851
MD     603
SC     534
DC     293
Name: State, dtype: int64